##### ----------------------Notes---------------------
* This code DOES NOT use regridded datafiles.

##### ------------------------------------------------

### 0. Set up Workspace

In [2]:
import sys
sys.path.append('../')

In [3]:
######################## Load modules ###################################
%matplotlib inline
import importlib
import time
import pickle
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rc

In [4]:
####################### Set up directory structure ######################
from plants_and_TCR.analysis_parameters import directory_information
from plants_and_TCR.analysis_parameters import get_CMIP_info
from plants_and_TCR.analysis_parameters import co2_concentrations
from plants_and_TCR.analysis_parameters import params
from plants_and_TCR.analyze_data import make_tcr_dataset
from plants_and_TCR.analyze_data import moving_average as movingaverage
from plants_and_TCR.process_data import calculate_tcr_time_series
from plants_and_TCR.process_data import calculate_pi_baseline
from plants_and_TCR.generate_figures import plot_scatter_CMIP

CDICT_NAMES=params.CDICT_NAMES
PATH_FIGURES = directory_information.DIR_OUTPUT_FIGURES
#PATH_DATA=directory_information.'/eos9/czarakas/Code/CMIP_Analysis/Data/Intermediate_Data/'

CO2_1PCTCO2 = co2_concentrations.co2_1pctco2
DEFAULT_VARNAME = params.DEFAULT_VARNAME
CDICT_NAMES = params.CDICT_NAMES
runnames_all = params.RUNNAMES_ALL
CMIP_linestyles = params.CMIP_LINESTYLES

In [5]:
rc('font',**{'family':'sans-serif','sans-serif':['Arial']})

### Calculate TCRs

In [ ]:
TCR_DICT = calculate_tcr_time_series.create_tcr_datasets(tcr_types=['TOT-RAD','TOT', 'RAD','PHYS'],
                                                         average_types=['global'],
                                                         save_tcr_dict=True,
                                                         recalculate_TCRs=False)

In [6]:
# Load precalculated
DIR_TCR_DICT = directory_information.DIR_TCR_DICT
TCR_DICT = pickle.load(open(DIR_TCR_DICT+'TCR_dict.pickle', "rb"))

In [7]:
TCR_2xco2_global = make_tcr_dataset.make_tcr_dataset(TCR_DICT, varname='tas', tcr_types=['TOT','RAD','TOT-RAD','PHYS'],
                                                     end_yr=70, average_type='global')
TCR_4xco2_global = make_tcr_dataset.make_tcr_dataset(TCR_DICT, varname='tas', tcr_types=['TOT','RAD','TOT-RAD','PHYS'],
                                                     end_yr=130, average_type='global')
TCR_2xco2_land = make_tcr_dataset.make_tcr_dataset(TCR_DICT, varname='tas', tcr_types=['TOT','RAD','TOT-RAD','PHYS'],
                                                   end_yr=70, average_type='land')
TCR_2xco2_ocean = make_tcr_dataset.make_tcr_dataset(TCR_DICT, varname='tas', tcr_types=['TOT','RAD','TOT-RAD','PHYS'],
                                                    end_yr=70, average_type='ocean')

In [8]:
TCR_4xco2_land = make_tcr_dataset.make_tcr_dataset(TCR_DICT, varname='tas', tcr_types=['TOT','RAD','TOT-RAD','PHYS'],
                                                   end_yr=130, average_type='land')
TCR_4xco2_ocean = make_tcr_dataset.make_tcr_dataset(TCR_DICT, varname='tas', tcr_types=['TOT','RAD','TOT-RAD','PHYS'],
                                                    end_yr=130, average_type='ocean')

In [9]:
TCR_2xco2_ocean = make_tcr_dataset.make_tcr_dataset(TCR_DICT, varname='tas', tcr_types=['TOT','RAD','TOT-RAD','PHYS'],
                                                    end_yr=70, average_type='ocean')
TCR_4xco2_ocean = make_tcr_dataset.make_tcr_dataset(TCR_DICT, varname='tas', tcr_types=['TOT','RAD','TOT-RAD','PHYS'],
                                                    end_yr=130, average_type='ocean')

In [10]:
TCR_2xco2_global['TOT-RAD_pct'] = TCR_2xco2_global['TOT-RAD']*100/TCR_2xco2_global['TOT']
TCR_4xco2_global['TOT-RAD_pct'] = TCR_4xco2_global['TOT-RAD']*100/TCR_4xco2_global['TOT']
TCR_2xco2_land['TOT-RAD_pct'] = TCR_2xco2_land['TOT-RAD']*100/TCR_2xco2_land['TOT']
TCR_4xco2_land['TOT-RAD_pct'] = TCR_4xco2_land['TOT-RAD']*100/TCR_4xco2_land['TOT']
TCR_2xco2_ocean['TOT-RAD_pct'] = TCR_2xco2_ocean['TOT-RAD']*100/TCR_2xco2_ocean['TOT']
TCR_4xco2_ocean['TOT-RAD_pct'] = TCR_4xco2_ocean['TOT-RAD']*100/TCR_4xco2_ocean['TOT']

TCR_2xco2_global_cmip5 = TCR_2xco2_global[0:8]
TCR_2xco2_global_cmip6 = TCR_2xco2_global[8:100]
TCR_4xco2_global_cmip5 = TCR_4xco2_global[0:8]
TCR_4xco2_global_cmip6 = TCR_4xco2_global[8:100]
TCR_2xco2_land_cmip6 = TCR_2xco2_land[8:100]
TCR_2xco2_ocean_cmip6 = TCR_2xco2_ocean[8:100]
TCR_4xco2_land_cmip6 = TCR_4xco2_land[8:100]
TCR_4xco2_ocean_cmip6 = TCR_4xco2_ocean[8:100]

# Calculate Numbers for In Text

In [11]:
model_order = ["bcc-csm1-1","CanESM2","CESM1-BGC","GFDL-ESM2M","HadGEM2-ES",
              "IPSL-CM5A-LR","NorESM1-ME","MPI-ESM-LR",
              "BCC-CSM2-MR","CanESM5","CESM2","GFDL-ESM4","UKESM1-0-LL",
              "IPSL-CM6A-LR","NorESM2-LM","MPI-ESM1-2-LR",
              "ACCESS-ESM1-5","CNRM-ESM2-1","GISS-E2-1-G","MIROC-ES2L"]
TCR_2xco2_global['RAD_pct'] = TCR_2xco2_global['RAD']*100/TCR_2xco2_global['TOT']
TCR_2xco2_global['PHYS_pct'] = TCR_2xco2_global['PHYS']*100/TCR_2xco2_global['TOT']
TCR_2xco2_global=TCR_2xco2_global.reindex(model_order)

print('-------')
cmip5_mean = TCR_2xco2_global[0:8].mean()
cmip6_mean = TCR_2xco2_global[8:100].mean()
cmip6_mean_samesubset = TCR_2xco2_global[8:16].mean()
cmip5_std = TCR_2xco2_global[0:8].std()
cmip6_std = TCR_2xco2_global[8:100].std()
cmip6_std_samesubset = TCR_2xco2_global[8:16].std()
print('-------')

TCR_2xco2_global.round(1)

-------
-------


,TOT,RAD,TOT-RAD,PHYS,TOT-RAD_pct,RAD_pct,PHYS_pct
bcc-csm1-1,1.7,1.7,0.1,0.0,3.5,96.5,2.8
CanESM2,2.3,2.2,0.2,0.2,7.9,92.1,6.5
CESM1-BGC,1.7,1.6,0.1,0.1,6.8,93.2,6.6
GFDL-ESM2M,1.4,1.3,0.0,0.0,0.1,99.9,3.4
HadGEM2-ES,2.5,2.0,0.5,0.4,20.1,79.9,14.6
IPSL-CM5A-LR,2.0,2.0,0.0,0.1,1.7,98.3,5.6
NorESM1-ME,1.6,1.4,0.1,0.1,9.2,90.8,5.3
MPI-ESM-LR,2.0,1.9,0.1,0.2,3.6,96.4,9.9
BCC-CSM2-MR,1.7,1.7,0.1,0.5,4.2,95.8,31.4
CanESM5,2.7,2.6,0.1,0.0,3.8,96.2,1.5


In [100]:
print(np.round(TCR_2xco2_land_cmip6['TOT-RAD_pct'].mean(),1))

7.8


In [108]:
TCR_2xco2_global_cmip6.std()

TOT            0.432033
RAD            0.396888
TOT-RAD        0.086499
PHYS           0.164677
TOT-RAD_pct    3.817511
dtype: float64

### ***Abstract and Results (3.1)***

#### All Models, 2xCO2
* In CMIP6, PHYS contributes A ± B° (range of C-D°) to the TCR
* corresponding to to E ± F% of the full TCR (and a range of G-H%)
* In CMIP5 models, PHY contributes I ± J°C (and a range of K-L°C) to the TCR
* corresponding to M ± N% of the full TCR (and a range of O-P%)

#### Same Model Subset, 2xCO2
* mean TCRPHYS across models remained roughly constant (Table S4).
* TCRPHYS increased for Q/R models

#### All Models, 4xCO2
* by 4xCO2, physiologically-forced warming reaches S-T° (U-V%) of warming (mean: W°, X%)

In [12]:
A = np.round(TCR_2xco2_global_cmip6['TOT-RAD'].mean(),2)
print('CMIP6 Mean TCRphys = '+str(A))
B = np.round(TCR_2xco2_global_cmip6['TOT-RAD'].std(),2)
print('CMIP6 Stddev TCRphys = '+str(B))
C = np.round(TCR_2xco2_global_cmip6['TOT-RAD'].min(),2)
D = np.round(TCR_2xco2_global_cmip6['TOT-RAD'].max(),2)
print('CMIP6 Range TCRphys = '+str(C)+' - '+str(D))

G = np.round(TCR_2xco2_global_cmip6['TOT-RAD_pct'].mean(),1) #Mean of Percents
#G = np.round(TCR_2xco2_global_cmip6['TOT-RAD'].mean()*100/TCR_2xco2_global_cmip6['TOT'].mean(),2) #Percents of Means
print('CMIP6 Mean % TCRphys = '+str(G))
F = np.round(TCR_2xco2_global_cmip6['TOT-RAD_pct'].std(),1)
print('CMIP6 Stddev % TCRphys = '+str(F))
G = np.round(TCR_2xco2_global_cmip6['TOT-RAD_pct'].min(),1)
H = np.round(TCR_2xco2_global_cmip6['TOT-RAD_pct'].max(),1)
print('CMIP6 Range % TCRphys = '+str(G)+'% - '+str(H)+'%')

CMIP6 Mean TCRphys = 0.12
CMIP6 Stddev TCRphys = 0.09
CMIP6 Range TCRphys = 0.02 - 0.29
CMIP6 Mean % TCRphys = 6.1
CMIP6 Stddev % TCRphys = 3.8
CMIP6 Range % TCRphys = 1.4% - 13.9%


In [13]:
A = np.round(TCR_4xco2_global_cmip6['TOT-RAD'].mean(),2)
print('4xCO2 CMIP6 Mean TCRphys = '+str(A))
B = np.round(TCR_4xco2_global_cmip6['TOT-RAD'].std(),2)
print('4xCO2 CMIP6 Stddev TCRphys = '+str(B))
C = np.round(TCR_4xco2_global_cmip6['TOT-RAD'].min(),2)
D = np.round(TCR_4xco2_global_cmip6['TOT-RAD'].max(),2)
print('4xCO2 CMIP6 Range TCRphys = '+str(C)+' - '+str(D))

G = np.round(TCR_4xco2_global_cmip6['TOT-RAD_pct'].mean(),2) #Mean of Percents
#G_ = np.round(TCR_4xco2_global_cmip6['TOT-RAD'].mean()*100/TCR_4xco2_global_cmip6['TOT'].mean(),2) #Percents of Means
print('4xCO2 CMIP6 Mean % TCRphys = '+str(G))
F = np.round(TCR_4xco2_global_cmip6['TOT-RAD_pct'].std(),2)
print('4xCO2 CMIP6 Stddev % TCRphys = '+str(F))
G = np.round(TCR_4xco2_global_cmip6['TOT-RAD_pct'].min(),2)
H = np.round(TCR_4xco2_global_cmip6['TOT-RAD_pct'].max(),2)
print('4xCO2 CMIP6 Range % TCRphys = '+str(G)+'% - '+str(H)+'%')

4xCO2 CMIP6 Mean TCRphys = 0.21
4xCO2 CMIP6 Stddev TCRphys = 0.12
4xCO2 CMIP6 Range TCRphys = 0.03 - 0.45
4xCO2 CMIP6 Mean % TCRphys = 4.94
4xCO2 CMIP6 Stddev % TCRphys = 2.45
4xCO2 CMIP6 Range % TCRphys = 0.51% - 7.93%


### ***Results (3.2)***

In [23]:
land_warming_2xCO2 = np.round(TCR_2xco2_land_cmip6['TOT-RAD'].mean(),2)
print('Mean land warming at 2xCO2 = '+str(land_warming_2xCO2))
land_warming_4xCO2 = np.round(TCR_4xco2_land_cmip6['TOT-RAD'].mean(),2)
print('Mean land warming at 4xCO2 = '+str(land_warming_4xCO2))
ocean_warming_2xCO2 = np.round(TCR_2xco2_ocean_cmip6['TOT-RAD'].mean(),2)
print('Mean ocean warming at 2xCO2 = '+str(ocean_warming_2xCO2))
ocean_warming_4xCO2 = np.round(TCR_4xco2_ocean_cmip6['TOT-RAD'].mean(),2)
print('Mean ocean warming at 4xCO2 = '+str(ocean_warming_4xCO2))

Mean land warming at 2xCO2 = 0.22
Mean land warming at 4xCO2 = 0.41
Mean ocean warming at 2xCO2 = 0.09
Mean ocean warming at 4xCO2 = 0.14


In [28]:
TCR_2xco2_land_cmip6[]

,TOT,RAD,TOT-RAD,PHYS
CNRM-ESM2-1,2.417871,2.383435,0.034434,-0.199386
BCC-CSM2-MR,2.509400,2.395498,0.113902,0.422824
CanESM5,3.740882,3.541230,0.199648,0.131675
CESM2,2.676840,2.209910,0.466927,0.163705
GISS-E2-1-G,2.506394,2.298412,0.207980,0.125303
UKESM1-0-LL,3.922472,3.375532,0.546940,0.521233
GFDL-ESM4,2.231402,2.165839,0.065552,0.304600
IPSL-CM6A-LR,3.179621,2.950991,0.228630,0.379856
MIROC-ES2L,2.195848,2.142798,0.053050,0.079192
NorESM2-LM,2.217801,2.017639,0.200162,NaN


In [40]:
land_phys_pct_2xCO2 = np.round(TCR_2xco2_land_cmip6['TOT-RAD_pct'].mean(),1)
print('Land % of total warming, 2xCO2: '+str(land_phys_pct_2xCO2)+'%')
ocean_phys_pct_2xCO2 = np.round(TCR_2xco2_ocean_cmip6['TOT-RAD_pct'].mean(),1)
print('Ocean % of total warming, 2xCO2: '+str(ocean_phys_pct_2xCO2)+'%')
land_phys_pct_4xCO2 = np.round(TCR_4xco2_land_cmip6['TOT-RAD_pct'].mean(),1)
print('Land % of total warming, 4xCO2: '+str(land_phys_pct_4xCO2)+'%')
ocean_phys_pct_4xCO2 = np.round(TCR_4xco2_ocean_cmip6['TOT-RAD_pct'].mean(),1)
print('Ocean % of total warming, 4xCO2: '+str(ocean_phys_pct_4xCO2)+'%')

Land % of total warming, 2xCO2: 7.8%
Ocean % of total warming, 2xCO2: 5.2%
Land % of total warming, 4xCO2: 6.8%
Ocean % of total warming, 4xCO2: 4.0%


In [44]:
TCR_2xco2_ocean_cmip6.round(2)

,TOT,RAD,TOT-RAD,PHYS,TOT-RAD_pct
CNRM-ESM2-1,1.61,1.55,0.05,-0.13,3.35
BCC-CSM2-MR,1.43,1.37,0.06,0.56,4.31
CanESM5,2.36,2.28,0.07,0.02,3.11
CESM2,1.81,1.59,0.22,0.09,12.22
GISS-E2-1-G,1.51,1.46,0.05,0.06,3.48
UKESM1-0-LL,2.37,2.19,0.18,0.05,7.73
GFDL-ESM4,1.37,1.36,0.01,0.13,0.62
IPSL-CM6A-LR,2.00,1.96,0.03,0.10,1.57
MIROC-ES2L,1.30,1.27,0.03,0.07,2.62
NorESM2-LM,1.22,1.11,0.11,NaN,8.79


### ***Results (3.3)***

In [ ]:
TCR_2xco2_land_cmip6['TOT'].std()

In [101]:
print('------Standard Deviation------')
pct_rad_explains = np.round(TCR_2xco2_global_cmip6['RAD'].std()*100/TCR_2xco2_global_cmip6['TOT'].std(),1)
print('Global: RAD accounts for '+str(pct_rad_explains)+'% of TOT standard deviation')
print('Global: TOT-RAD accounts for remaining '+str(round(100-pct_rad_explains,1))+'% of TOT standard deviation')
pct_rad_explains = np.round(TCR_2xco2_land_cmip6['RAD'].std()*100/TCR_2xco2_land_cmip6['TOT'].std(),1)
print('Land: RAD accounts for '+str(pct_rad_explains)+'% of TOT standard deviation')
print('Land: TOT-RAD accounts for remaining '+str(round(100-pct_rad_explains,1))+'% of TOT standard deviation')

------Standard Deviation------
Global: RAD accounts for 91.9% of TOT standard deviation
Global: TOT-RAD accounts for remaining 8.1% of TOT standard deviation
Land: RAD accounts for 86.4% of TOT standard deviation
Land: TOT-RAD accounts for remaining 13.6% of TOT standard deviation


In [103]:
TCR_2xco2_land_cmip6.mean()

TOT            2.743394
RAD            2.520258
TOT-RAD        0.223135
PHYS           0.230471
TOT-RAD_pct    7.812362
dtype: float64

In [102]:
print('------Inter-Model Variation------')

inter_model_range = ((TCR_2xco2_global_cmip6.max() - TCR_2xco2_global_cmip6.min())*100/(TCR_2xco2_global_cmip6['TOT'].max() - TCR_2xco2_global_cmip6['TOT'].min()))
print('------Global------')
print(inter_model_range)

inter_model_range = ((TCR_2xco2_land_cmip6.max() - TCR_2xco2_land_cmip6.min())*100/(TCR_2xco2_land_cmip6['TOT'].max() - TCR_2xco2_land_cmip6['TOT'].min()))
print('------Land------')
print(inter_model_range)

------Inter-Model Variation------
------Global------
TOT            100.000000
RAD             98.572502
TOT-RAD         20.141908
PHYS            52.688652
TOT-RAD_pct    953.860842
dtype: float64
------Land------
TOT            100.000000
RAD             88.241032
TOT-RAD         29.682570
PHYS            41.735747
TOT-RAD_pct    927.770644
dtype: float64


#### All Models, 2xCO2

In CMIP6 models, the physiological effect contributes A°C (standard deviation: B°C; range: C-D°C) of warming to the TCR, corresponding to E% of the full TCR (standard deviation: F%; range: G-H%). 

* In CMIP5 models, PHY contributes I ± J°C (and a range of K-L°C) to the TCR
* corresponding to M ± N% of the full TCR (and a range of O-P%)

#### Same Model Subset, 2xCO2
* mean TCRPHYS across models remained roughly constant (Table S4).
* TCRPHYS increased for Q/R models

#### All Models, 4xCO2
* by 4xCO2, physiologically-forced warming reaches S-T° (U-V%) of warming (mean: W°, X%)